<a href="https://colab.research.google.com/github/forgef/LDC/blob/master/2_EU_Data_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reconstruction of the EU tariffs under preferential treatment**



Data for tariffs imposed on exporters by the European Union are not available on the [WTO website](https://timeseries.wto.org/). In this notebook, I use official European laws to reconstruct the time series data from EU official documents available in PDFs.


# SET-UP

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install tabula-py
!pip install PyPDF2
!pip install tika
!pip install Unidecode

     |████████████████████████████████| 10.4MB 3.0MB/s 
     |████████████████████████████████| 81kB 2.4MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=631e192ac517705afdb50ad67aeade00ce84475bb1df1a0b470a7261a716eefc
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2
  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=d201a497c8b9338d25d1f0345f809c7d198c3e1ea55f95a6589b82437e88db72
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika
     |████████████████████████████████| 245kB 3.4MB/s 


In [3]:
# Libraries
import numpy as np
import pandas as pd
import tabula as tb
from PyPDF2 import PdfFileWriter, PdfFileReader
from tika import parser 
from bs4 import BeautifulSoup
import requests
import os
import re
import unidecode

# Data path
dataFolder='/content/gdrive/My Drive/LDC/Data/'

# Most Favored Nation from WTO

In [ ]:
dd=pd.read_csv('/content/gdrive/My Drive/LDC/Data/European Union_MFN.csv')
print("Years available:",', '.join([str(y) for y in dd.year.unique()]))
print('Number of codes per year:')
display(dd.groupby('year',as_index=False)['HS6'].count())
dd.head()

Years available: 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2004, 2007, 2009, 2013, 2015, 2016, 2017
Number of codes per year:


,year,HS6
0,1996,5113
1,1997,5113
2,1998,5113
3,1999,5113
4,2000,5113
5,2001,5113
6,2002,5224
7,2004,5224
8,2007,5052
9,2009,5051


,importerCode,importer,HS6,year,preference,tariff_pct
0,918,European Union,10391,1996,MFN,14.8
1,918,European Union,10593,1996,MFN,16.1
2,918,European Union,10599,1996,MFN,11.6
3,918,European Union,20120,1996,MFN,108.7
4,918,European Union,20220,1996,MFN,79.1



# DATA MINING: extract data from PDF files.
There are three sources of tariff data that will be extracted from legal documents: <br>

1. __ACP-EEC convention__ of 5 March 1990 on the arrangements applicable to agricultural products and certain goods resulting from the processing of agricultural products originating in the ACP States or in the overseas countries and territories (OCT) from the [COUNCIL REGULATION (EEC) No 715/90](https://op.europa.eu/en/publication-detail/-/publication/f804b179-db24-472d-af50-abd255af0b5d)
2. __Scheme of generalised tariff preferences__ for the period 1 July 1999 to 31 December 2001 from [COUNCIL REGULATION (EC) No 2820/98](https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:31998R2820&from=en)

3. __Scheme of generalised tariff preferences__ for the period from 1 January 2002 to 31
December 2004 from [COUNCIL REGULATION (EC) No 2501/2001](https://eur-lex.europa.eu/resource.html?uri=cellar:639b8f1a-836e-4eda-ad98-d0ae3839900b.0004.02/DOC_1&format=PDF)

In [ ]:
# save urls in dictionnary
urls={'eu_1989':'https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:31989R0571&qid=1590343851250&from=EN',
    'eu_1990':'https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:31990R0715&from=EN',
    'eu_1998':'https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:31998R2820&from=en',
   'eu_2001':'https://eur-lex.europa.eu/resource.html?uri=cellar:639b8f1a-836e-4eda-ad98-d0ae3839900b.0004.02/DOC_1&format=PDF' 
}
# Get pdf from EU's wesites
for u in urls.keys():
  pathPDF=''.join([dataFolder,u,'.pdf'])
  if not os.path.isfile(pathPDF): 
    myfile = requests.get(urls[u])
    print(f'Saving {u}')
    open(pathPDF, 'wb').write(myfile.content)
  else:
    print(f"{u} PDF already exists.")  

#Split PDF in single pages
for euPdf in ['eu_1990','eu_1998']:
  inputpdf = PdfFileReader(open(''.join([dataFolder,euPdf,'.pdf']), "rb"))
  for i in range(inputpdf.numPages) :
      output = PdfFileWriter()
      output.addPage(inputpdf.getPage(i))
      with open(''.join([dataFolder,euPdf,'_',str(i+1),'.pdf']), "wb") as outputStream:
          output.write(outputStream)

eu_1989 PDF already exists.
eu_1990 PDF already exists.
eu_1998 PDF already exists.
eu_2001 PDF already exists.


## EU 1990 Tariffs (Lomé accords with ACP countries)


### Beneficiaries
__Article 1.1__ 

_This Regulation shall apply to products originating
in the ACP States listed in Annex I or in the countries and territories listed in Annex II._

__Article 15__

Lists goods that shall be imported free of customs duties.

Beneficiary countries in Annex I p22


In [ ]:
# instantiate list of beneficiaries
ben=[]
# Text to ignore
notCountry = ["^No","^ANNEX","^LIST"]
# Make a regex that matches if any of my regexes match.
combined = "(" + ")|(".join(notCountry) + ")"
# Read pdf to lxml
raw=parser.from_file(''.join([dataFolder,'eu_1990_22.pdf']),xmlContent=True)['content']
txt=BeautifulSoup(raw,'lxml')
#iterate over paragraphes
rows = txt.find_all('p')
for r in rows:
  # Iterate over country names
  countries=r.get_text().splitlines()
  for country in countries:
    # Clean countries that contain accents
    if not re.match(combined, country):
      if country in ["Cdte-d'Ivoire","Sio Tom6 and Principe"]:
        country=country.replace("Cdte-d'Ivoire","Cote-d'Ivoire")
        country=country.replace("Sio Tom6 and Principe","Sao Tome and Principe")
      if country != '':
        # append country to list
        ben.append(country.upper())
# Save in a DF        
df=pd.DataFrame({'beneficiary_eu_1990':np.sort(ben)})
# To CSV
df.to_csv(''.join([dataFolder, 'beneficiaries_eu_1990.csv']),index=False)
print(f"There {len(df)} beneficiaries listed.")
df.head()

There 68 beneficiaries listed.


,beneficiary_eu_1990
0,ANGOLA
1,ANTIGUA AND BARBUDA
2,BAHAMAS
3,BARBADOS
4,BELIZE


## Eu 1998 Tariffs (GSP - LDC)

### Beneficiaries

__Annex III__ p 80: "_List of beneficiary countries and territories enjoying generalised tariff preferences_" <br>

If dataframe contains in comment:<br>
_(1) Preferences for these countries are limited to the agricultural products in Chapters 1 to 24 of the Combined Nomenclature listed in Annex I._<br>
_(2) This country is also included in Annex IV._<br>
_(3) This country is also included in Annex V._<br>

__Annex IV__ p82: 
"_List of least-developed developing countries_" <br>

__Annex V__ p82: 
ANNEX V

_List of countries referred to in [Article 7](#article7)_ <br>

Andean Group<br>
CO Colombia<br>
VE Venezuela<br>
EC Ecuador<br>
PE Peru<br>
BO Bolivia<br>

Central American Common Market<br>
GT Guatemala<br>
HN Honduras<br>
SV El Salvador<br>
NI Nicaragua<br>
CR Costa Rica<br>
PA Panama


In [ ]:
print('Starting Annexes III')

ben=[]
notCountry = ["^L 357/80","^ANNEX","^List of","^A\. INDEPENDENT",r"\(\*\) THE"]
# Make a regex that matches if any of my regexes match.
combined = "(" + ")|(".join(notCountry) + ")"
pdfFile=''.join(['eu_1998_80','.pdf'])
# Parse PDF
raw=parser.from_file(''.join([dataFolder,pdfFile]),xmlContent=True)['content']
txt=BeautifulSoup(raw,'lxml')
# Find paragraphs
rows = txt.find_all('p')
for r in rows:
  # Iterate over country names
  countries=r.get_text().splitlines()
  for country in countries:
    # Clean countries that contain accents
    if not re.match(combined, country):
      if country != '':
        if country=='XM Former Yugoslav Republic of':
          break
        else:
          if country=='Macedonia (1)':
            country='XM MACEDONIA (1)'
        # append country to list
        ben.append(unidecode.unidecode(country.upper()))
        
# Save in a DF        
ben=ben[:-5]
ben.sort()
df=pd.DataFrame({'beneficiary_1998':ben})
df[['beneficiary_1998','comment']]=df.beneficiary_1998.str.split('(\([0-9]\))$',n=1,expand=True).iloc[:,0:2]
df['beneficiary_1998']=df.beneficiary_1998.str.split(' ',n=1).str[1]
df['beneficiary_1998']=df['beneficiary_1998'].astype(str).str.rstrip()
#Correct typo:
df.loc[df.beneficiary_1998=='ETHIOPA','beneficiary_1998']='ETHIOPIA'
df['GSP']=1
print(f"\nThe data contains {len(df)} countries including {len(df.loc[df.comment=='(2)'])} from LDC. ")

print("\n\n")
print('Starting Annexes IV and V')
ben=[]
notCountry = ["^L 357/82","^ANNEX","^List of","^A\. INDEPENDENT",r"\(\*\) THE"]
p = re.compile("^\w\w\s(.*)")

# Make a regex that matches if any of my regexes match.
combined = "(" + ")|(".join(notCountry) + ")"
pdfFile=''.join(['eu_1998_82','.pdf'])
# Parse PDF
raw=parser.from_file(''.join([dataFolder,pdfFile]),xmlContent=True)['content']
txt=BeautifulSoup(raw,'lxml')
# Find paragraphs
rows = txt.find_all('p')
for r in rows:
  # Iterate over country names
  countries=r.get_text().splitlines()
  for country in countries:
    # Clean countries that contain accents
    if not re.match(combined, country):
      if country != '':
        if country=='XM Former Yugoslav Republic of':
          break
        else:
          if country=='Macedonia (1)':
            country='XM MACEDONIA (1)'
        # append country to list
        country=unidecode.unidecode(country.upper())
        try:
          ben.append(p.match(country).group(1))
        except AttributeError:
          pass
          
ldc=ben[:ben.index('SAMOA') +1]
ldc = ['MYANMAR (FORMERLY BURMA)' if x=='MYANMAR (FORMERLY' else x for x in ldc]

other=ben[ben.index('SAMOA') +1:]

df['LDC']=(df.beneficiary_1998.isin(ldc)).astype(int)
df['A7']=(df.beneficiary_1998.isin(other)).astype(int)
df.loc[~df.comment.isna(),'GSP']=0

print(f"There are {len(ldc)} countries in LDC and {len(df.loc[df.comment=='(2)'])} counted as LDC in the Annex III.\n\
There are {len(other)} countries in Article 7 and {len(df.loc[df.comment=='(3)'])} counted as Article 7 in the Annex III.")
df.pop('comment')
df.head()

2020-06-30 18:16:16,067 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.


Starting Annexes III


2020-06-30 18:16:17,355 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2020-06-30 18:16:17,722 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...



The data contains 146 countries including 48 from LDC. 



Starting Annexes IV and V
There are 48 countries in LDC and 48 counted as LDC in the Annex III.
There are 11 countries in Article 7 and 11 counted as Article 7 in the Annex III.


,beneficiary_1998,GSP,LDC,A7
0,UNITED ARAB EMIRATES,1,0,0
1,AFGHANISTAN,0,1,0
2,ANTIGUA AND BARBUDA,1,0,0
3,ALBANIA,1,0,0
4,ARMENIA,1,0,0


### Tariffs

__Article 1.2__ <br>

p4. This Regulation shall apply to products falling within
Chapters 1 to 97 of the Common Customs Tariff,
excluding Chapter 93, which are listed in <font color='red'>Annex I</font>. It shall only apply to the products listed in Annex VII on
the conditions laid down in Articles 6 and 7 <br>

__Article 2__ <br>
 The preferential duty applying to products listed in
_Part 1 of Annex I_ shall be 85 % of the Common
Customs Tariff duty applicable to the product concerned, without prejudice to the provisions of Title II.<br>
The preferential duty applying to products listed in _Part 2 of Annex I_ shall be 70 % of the Common
Customs Tariff duty applicable to the product concerned, without prejudice to the provisions of Title II. <br>
The preferential duty applying to products listed in _Part 3 of Annex I_ shall be 35 % of the Common
Customs Tariff duty applicable to the product concerned,
without prejudice to the provisions of Title II.
Common Customs Tariff duties shall be suspended in
their entirety on products listed in _Part 4 of Annex I_.

__Article 6__ <br>
For the least-developed countries listed in annex IV, Common Customs Tariff duties shall be suspended in their <font color='red'>entirety</font> in respect of the products listed in Annex I and <font color='red'>reduced</font> according to the modulation mechanism provided for in <font color='red'>Article 2</font> in respect of the products listed in <font color='red'>Annex VII</font>. <br>

<font color='red'>My interpretation:</font> <font color='blue'>For LDC countries, any good listed in Annex I should be zero. If a good is listed in Annex VII then tariffs should be 85% of MFN for part 1, 70% of MFN for part 2, 35% of MFN for part 3 and 0% for part 4. </font> <br>

__Article 7__ <a id='article7'></a>
<br>
For the countries listed in <font color='red'>Annex V</font>, Common Customs
Tariff duties shall be suspended in their entirety on the industrial products listed in Annex I falling within Chapters 25 to 97 of the Common Customs Tariff, excluding Chapter 93, and on the agricultural products listed in Annex VII, except for those marked with an asterisk and without prejudice to the procedure described
in Article 31(3). <br>




__Annex I__ <br>
Part 1: p15-28 <br>
Part 2: p29-47 <br>
Part 3: p48-62 <br>
Part 4: p63-72 <br>

__Annex VII__ <br>
Part 1: p84-90 <br>
Part 2: p91-92 <br>
Part 3: p92 <br>
Part 4: p93-107 <br>

In [ ]:
parts={'Part 1':range(15,29),'Part 2':range(29,48),'Part 3': range(48,63),'Part 4': range(63,73)}
parts2={'Part 1':range(84,91),'Part 2':range(91-92),'Part 3': 92,'Part 4':range(93,108)}

In [ ]:
# Create DataFrame list:
DFs=[]
#Loop over the 4 parts
for i, v in enumerate(parts.keys()):
  # Set variable lists
  hs_code=[] ; hs_level=[] ; comments=[]
  for page in parts[v]:

    pdfFile=''.join(['eu_1998_',str(page),'.pdf'])
    # Parse PDF
    raw=parser.from_file(''.join([dataFolder,pdfFile]),xmlContent=True)['content']
    txt=BeautifulSoup(raw,'lxml')

    # Find paragraphs
    rows = txt.find_all('p')
    #Iterate over rows
    for s in rows:
      s=s.get_text()
      # Print out the Part
      if re.match(r"(?i)^Part",s):
        print(s)
      # Deal with exceptions
      if re.match(r"(?i)^ex",s):
        try:
          ss=re.search('(?i)[0-9]{1,4}(\s[0-9]{2}){0,2}', s).group()
          hs=ss.replace(" ","")
          # Append to lists
          hs_code.append(hs) ; hs_level.append(len(hs)) ; comments.append(s.split(ss)[1])
        except AttributeError:
          pass
      elif re.match(r"(?i)[0-9]{1,4}(\s[0-9]{2}){0,2}",s):
        hs=re.search('(?i)[0-9]{1,4}(\s[0-9]{2}){0,2}', s).group().replace(" ","")         
        # Append to lists
        hs_code.append(hs) ; hs_level.append(len(hs)) ; comments.append(np.nan)
      else:
        pass

  assert len(hs_code)==len(hs_level) ; assert len(hs_code)==len(comments)

  df=pd.DataFrame({'HScode':hs_code, 'HSlevel':hs_level, 'comment':comments})
  i+=1
  df['part']= np.full((len(df), 1), i, dtype=np.int8) 

  display(df.info())
  DFs.append(df)
  
df=pd.concat(DFs)
df.reset_index(inplace=True, drop=True)
df.head().append(df.tail())

PART 1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   HScode   288 non-null    object
 1   HSlevel  288 non-null    int64 
 2   comment  16 non-null     object
 3   part     288 non-null    int8  
dtypes: int64(1), int8(1), object(2)
memory usage: 7.2+ KB


None

PART 2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   HScode   422 non-null    object
 1   HSlevel  422 non-null    int64 
 2   comment  11 non-null     object
 3   part     422 non-null    int8  
dtypes: int64(1), int8(1), object(2)
memory usage: 10.4+ KB


None

PART 3

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   HScode   296 non-null    object
 1   HSlevel  296 non-null    int64 
 2   comment  16 non-null     object
 3   part     296 non-null    int8  
dtypes: int64(1), int8(1), object(2)
memory usage: 7.4+ KB


None

PART 4

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   HScode   155 non-null    object
 1   HSlevel  155 non-null    int64 
 2   comment  18 non-null     object
 3   part     155 non-null    int8  
dtypes: int64(1), int8(1), object(2)
memory usage: 3.9+ KB


None

,HScode,HSlevel,comment,part
0,01012010,8,NaN,1
1,03019190,8,NaN,1
2,03021190,8,NaN,1
3,03032190,8,NaN,1
4,03041011,8,NaN,1
1156,94038000,8,NaN,4
1157,940390,6,NaN,4
1158,940600,6,NaN,4
1159,95,2,"Toys, games and sports requisites; parts and ...",4
1160,96,2,"Miscellaneous manufactures articles, excludin...",4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   HScode   1161 non-null   object
 1   HSlevel  1161 non-null   int64 
 2   comment  61 non-null     object
 3   part     1161 non-null   int8  
dtypes: int64(1), int8(1), object(2)
memory usage: 28.5+ KB


## EU 2001 Tariffs (GSP)


### Beneficiaries
p2 of the PDF: "_The beneficiary countries of each of the arrangements referred
to in Article 1(2) are listed in Annex I._" <br>

__Beneficiary countries and territories of the Community's scheme of generalised tariff preferences__<br>
__Column A__: code according to the nomenclature of countries and territories for the external trade statistics of the
Community<br>
__Column B__: name of country<br>
__Column C__: sectors not included in the general arrangements for the beneficiary country concerned (Article 7(7))<br>
__Column D__: sectors in respect of which tariff preferences have been removed for the beneficiary country concerned
(Article 7(8))<br>
__Column E__: countries included in the special incentive arrangements for the protection of labour rights (Title III
Section 1)<br>
__Column F__: sectors included in these arrangements for the beneficiary country concerned (Article 8(1) and (2))<br>
Column G: countries included in the special incentive arrangements for the protection of the environment (Title III
Section 2)<br>
__Column H__: countries included in the special arrangements for least developed countries (Article 9)<br>
__Column I__: countries included in the special arrangements to combat drug production and trafficking (Title IV)

In [ ]:
# Extract table for beneficiaries
tabs=tb.read_pdf(dataFolder+'eu_2001.pdf',pages='13-18', encoding='latin_1',lattice=True)
ben=[]
for i in range(1,len(tabs)):
  if len(tabs[i].columns)>1:
    ben.append(tabs[i])
    print(len(tabs[i].columns))
ben=pd.concat(ben)
ben.rename(columns={'B':'exporter'},inplace=True)
display(ben.head().append(ben.tail()))
display(ben.info())
ben.to_csv(''.join([dataFolder,'beneficiaries_eu_2001.csv']))

Got stderr: May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
May 24, 2020 6:48:52 PM org.apache.pdfbox.pdmodel.fo

7
7
7
7
7
7


,exporter,C,D,E,F,G,H
0,UnitedArabEmirates,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,X
2,AntiguaandBarbuda,NaN,NaN,NaN,NaN,NaN,NaN
3,Anguilla,NaN,NaN,NaN,NaN,NaN,NaN
4,Armenia,"II,XXVI",NaN,NaN,NaN,NaN,NaN
16,Yemen,NaN,NaN,NaN,NaN,NaN,X
17,Mayotte,NaN,NaN,NaN,NaN,NaN,NaN
18,SouthAfrica,XXVI,NaN,NaN,NaN,NaN,NaN
19,Zambia,NaN,NaN,NaN,NaN,NaN,X
20,Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 0 to 20
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   exporter  179 non-null    object 
 1   C         15 non-null     object 
 2   D         20 non-null     object 
 3   E         1 non-null      object 
 4   F         1 non-null      object 
 5   G         0 non-null      float64
 6   H         49 non-null     object 
dtypes: float64(1), object(6)
memory usage: 11.2+ KB


None

### Tariffs




# RECONSTRUCTION OF TIME SERIES

## Timelime

__1971:__ The Community has granted trade preferences to developing countries, in the framework of its scheme of generalised tariff preferences.
Preferences were with respect to certain agricultural and industrial products from developing countries. The initial 10-year period of application of the system of preferences ended on 31 December 1980. A second 10-year period ended on 31 December 1990. The scheme was kept in being, unchanged, until 31 December 1994 when the Community made a new 10-year offer (1995-2004). source:  [OJ L 357, 30.12.1998,](https://eur-lex.europa.eu/LexUriServ/LexUriServ.do?uri=OJ:L:1998:357:0001:0112:EN:PDF)

### 1998
From COUNCIL REGULATION (EC) No [2820/98](https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:31998R2820&from=EN) of 21 December 1998 applying a multiannual scheme of generalised tariff preferences for the period 1 July 1999 to 31 December 2001
<br>

#### For LDC
__Section 3__
Special support arrangements for the least-developed
countries<br>

__Article 6__<br>
For the least-developed countries listed in _annex IV_ ,
Common Customs Tariff duties shall be suspended in
their entirety in respect of the products listed in __Annex I__
and reduced according to the modulation mechanism
provided for in __Article 2__ in respect of the products listed
in __Annex VII__. <br>

#### Tariffs for GSP (zero for LDC)
__Article 2__
1. The preferential duty applying to products listed in Part 1 of Annex I shall be 85 % of the Common Customs Tariff duty applicable to the product concerned, without prejudice to the provisions of Title II.
2. The preferential duty applying to products listed in Part 2 of Annex I shall be 70 % of the Common Customs Tariff duty applicable to the product concerned, without prejudice to the provisions of Title II.
3. The preferential duty applying to products listed in Part 3 of Annex I shall be 35 % of the Common Customs Tariff duty applicable to the product concerned, without prejudice to the provisions of Title II.
4. Common Customs Tariff duties shall be suspended in their entirety on products listed in Part 4 of Annex I.



